In [ ]:
import pandas as pd
import re
import geonamescache
import pycountry
import country_converter as coco
from nltk.tokenize import RegexpTokenizer

filename = ""
db = pd.read_csv(filename, index_col=False, error_bad_lines=False, engine='python',encoding="utf-8")
gc = geonamescache.GeonamesCache()

In [ ]:

def removepunctuation(word):
    tokenizer = RegexpTokenizer(r'\w+')
    word = tokenizer.tokenize(str(word))
    return re.sub(r'[^a-zA-Z\s]','',str(word))

def countryCode2CountryName(country_code):
    if country_code == 'None':
        return 'None'
    country = pycountry.countries.get(alpha_2=country_code)
    if country==None:
        cc = coco.CountryConverter()
        country_name = cc.convert(names = country_code, to = 'name_short')
        if country_name:
            return country_name
        else:
            return 'None'
    return country.name

def findCountry(location):
    location = removepunctuation(location)
    if location:
        locationInfo = gc.search_cities(location,case_sensitive=True)
        length = len(locationInfo)
        if(length>0):
            locationInfo = sorted(locationInfo, key=lambda e: e.__getitem__('population'))
            country = locationInfo[length-1]['countrycode']
            return country
    return 'None'
    
def dealWithLocation(info):
    info = str(info)
    words = info.split(',')
    for word in words:
        location = findCountry(word)
        if location!='None':
            return location
    return location

def deleteNoneType(db,rowName):
    df = db[(db[rowName]!='None')]
    return df



In [ ]:
# map user_location to the related country
locationList2 = list(db['user_location'])
db['country_code'] = list(map(lambda x: dealWithLocation(x), locationList2))
df_filterLocation = deleteNoneType(db,'country_code')

df_filterLocation['country_name'] = list(map(lambda x:countryCode2CountryName(x),df_filterLocation['country_code']))
df_filterLocation = deleteNoneType(df_filterLocation,'country_name')


In [ ]:
# write to new csv
def writeCsv(filename,data):
    dataframe = pd.DataFrame(data)
    dataframe.to_csv(filename,index=False,sep=',')

df_filterLocation.drop_duplicates(subset='text',inplace=True)
filename = ''
writeCsv(filename,df_filterLocation)

